# 🎂 Train Your Custom Cake AI Model

This notebook trains a custom AI model on your cake images and uploads it to Hugging Face.

## Before You Start:
1. ✅ Have 50+ cake images in `Google Drive > cake_training_images`
2. ✅ Create Hugging Face account at [huggingface.co/join](https://huggingface.co/join)
3. ✅ Create token at [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

## Setup:
- Go to **Runtime → Change runtime type → T4 GPU**
- Run cells in order

---

In [ ]:
# ===== CELL 1: Install Dependencies =====
!pip install diffusers transformers accelerate safetensors -q
!pip install peft bitsandbytes datasets -q
!pip install huggingface_hub -q
print("✅ Libraries installed!")

In [ ]:
# ===== CELL 2: Mount Google Drive =====
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

In [ ]:
# ===== CELL 3: YOUR CONFIGURATION =====
# ⚠️ CHANGE THESE VALUES!

HF_USERNAME = "your_username"  # Your Hugging Face username
HF_TOKEN = "hf_xxxxxxxxxx"      # Your token (with Write access)
MODEL_NAME = "cake-ai-model"      # Your model repo name

# Training images location
TRAINING_IMAGES = "/content/drive/MyDrive/cake_training_images"

# Output folder
OUTPUT_DIR = "/content/trained_cake_model"

print(f"📋 Configuration:")
print(f"   Username: {HF_USERNAME}")
print(f"   Model: {MODEL_NAME}")
print(f"   Training images: {TRAINING_IMAGES}")

In [ ]:
# ===== CELL 4: Check Training Images =====
import os

if not os.path.exists(TRAINING_IMAGES):
    print("❌ ERROR: Training images folder not found!")
    print(f"   Expected: {TRAINING_IMAGES}")
    print("   Please run the Generate_Cake_Images notebook first.")
else:
    images = [f for f in os.listdir(TRAINING_IMAGES) 
              if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"✅ Found {len(images)} training images!")
    print(f"   First 5: {images[:5]}")

In [ ]:
# ===== CELL 5: Login to Hugging Face =====
from huggingface_hub import login
login(token=HF_TOKEN)
print("✅ Logged in to Hugging Face!")

In [ ]:
# ===== CELL 6: Clone Training Scripts =====
%cd /content
!git clone https://github.com/huggingface/diffusers.git
%cd diffusers
!pip install -e . -q
%cd examples/dreambooth
!pip install -r requirements.txt -q
print("\n✅ Training scripts ready!")

In [ ]:
# ===== CELL 7: Configure Accelerate =====
from accelerate.utils import write_basic_config
write_basic_config()
print("✅ Accelerate configured!")

In [ ]:
# ===== CELL 8: START TRAINING! =====
# This will take 30-60 minutes depending on image count

import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Training command
!accelerate launch train_dreambooth_lora.py \
  --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
  --instance_data_dir="{TRAINING_IMAGES}" \
  --output_dir="{OUTPUT_DIR}" \
  --instance_prompt="a photo of sks cake" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --max_train_steps=800 \
  --seed=42 \
  --checkpointing_steps=200

print("\n🎉 Training complete!")

In [ ]:
# ===== CELL 9: Test Your Trained Model =====
import torch
from diffusers import StableDiffusionPipeline
from IPython.display import display

print("🧪 Testing trained model...")

# Load base model
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")

# Load your trained LoRA weights
pipe.load_lora_weights(OUTPUT_DIR)

# Test prompts
test_prompts = [
    "a photo of sks cake, chocolate with roses",
    "a photo of sks cake, wedding 3 tier white",
    "a photo of sks cake, birthday with sprinkles",
]

print("\n🎂 Generating test cakes...\n")
for i, prompt in enumerate(test_prompts):
    image = pipe(prompt, num_inference_steps=30).images[0]
    image.save(f"test_cake_{i+1}.png")
    print(f"Test {i+1}: {prompt}")
    display(image)
    print("")

In [ ]:
# ===== CELL 10: Save Full Model for Upload =====
FULL_MODEL_DIR = "/content/full_cake_model"

print("💾 Saving full model...")
pipe.save_pretrained(FULL_MODEL_DIR)
print(f"✅ Model saved to: {FULL_MODEL_DIR}")

In [ ]:
# ===== CELL 11: Upload to Hugging Face =====
from huggingface_hub import HfApi, create_repo

repo_id = f"{HF_USERNAME}/{MODEL_NAME}"

print(f"📤 Uploading to: {repo_id}")
print("⏳ This may take 5-10 minutes...\n")

# Create repo if doesn't exist
try:
    create_repo(repo_id, private=True, exist_ok=True)
    print(f"✅ Repository ready: {repo_id}")
except Exception as e:
    print(f"Repository may already exist: {e}")

# Upload
api = HfApi()
api.upload_folder(
    folder_path=FULL_MODEL_DIR,
    repo_id=repo_id,
    repo_type="model"
)

print(f"\n🎉 SUCCESS! Model uploaded!")
print(f"\n🔗 Your model: https://huggingface.co/{repo_id}")
print(f"🔗 API URL: https://router.huggingface.co/hf-inference/models/{repo_id}")

In [ ]:
# ===== CELL 12: Backup to Google Drive =====
import shutil

backup_path = f"/content/drive/MyDrive/{MODEL_NAME}_backup"

if os.path.exists(backup_path):
    shutil.rmtree(backup_path)

shutil.copytree(OUTPUT_DIR, backup_path)

print(f"✅ Model backed up to Google Drive!")
print(f"📁 Location: {backup_path}")

---
# ✅ All Done!

## What You Got:

1. **Trained Model** uploaded to Hugging Face
2. **Backup** saved to Google Drive

## Next Step - Update Your PHP:

In `generate_cake.php`, change line 9 to:

```php
$API_URL = "https://router.huggingface.co/hf-inference/models/YOUR_USERNAME/cake-ai-model";
```

And update the prompt (line 47):

```php
$finalPrompt = "a photo of sks cake, " . $customerPrompt;
```

---
**Your cake AI is now ready!** 🎂🤖